# **IRB Retail Mortgage**
## **Data Preparation**
At a high level, data preparation for a retail mortgage PD model typically includes:

- Data ingestion
- Initial filtering & portfolio definition
- Target (default flag) construction
- Missing value treatment
- Outlier treatment
- Feature transformations
- Train / validation split (time-based)

## **Step 1 - Data Ingestion**
Importing the retail mortgage dataset [Check Data Description](../README.md)

In [1]:
# importing the required modules
import sys
sys.path.append("..")
from src.config import *

In [2]:
# loading the raw mortgage dataset
mortgage_raw = pd.read_csv(f"{main_dir}/data/raw/accepted_2007_to_2018Q4.csv",low_memory=False)
mortgage_raw.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0000,3600.0000,3600.0000,36 months,13.9900,123.0300,C,C4,leadman,10+ years,MORTGAGE,55000.0000,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.9100,0.0000,Aug-2003,675.0000,679.0000,1.0000,30.0000,NaN,7.0000,0.0000,2765.0000,29.7000,13.0000,w,0.0000,0.0000,4421.7239,4421.7200,3600.0000,821.7200,0.0000,0.0000,0.0000,Jan-2019,122.6700,NaN,Mar-2019,564.0000,560.0000,0.0000,30.0000,1.0000,Individual,NaN,NaN,NaN,0.0000,722.0000,144904.0000,2.0000,2.0000,0.0000,1.0000,21.0000,4981.0000,36.0000,3.0000,3.0000,722.0000,34.0000,9300.0000,3.0000,1.0000,4.0000,4.0000,20701.0000,1506.0000,37.2000,0.0000,0.0000,148.0000,128.0000,3.0000,3.0000,1.0000,4.0000,69.0000,4.0000,69.0000,2.0000,2.0000,4.0000,2.0000,5.0000,3.0000,4.0000,9.0000,4.0000,7.0000,0.0000,0.0000,0.0000,3.0000,76.9000,0.0000,0.0000,0.0000,178050.0000,7746.0000,2400.0000,13734.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0000,24700.0000,24700.0000,36 months,11.9900,820.2800,C,C1,Engineer,10+ years,MORTGAGE,65000.0000,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.0600,1.0000,Dec-1999,715.0000,719.0000,4.0000,6.0000,NaN,22.0000,0.0000,21470.0000,19.2000,38.0000,w,0.0000,0.0000,25679.6600,25679.6600,24700.0000,979.6600,0.0000,0.0000,0.0000,Jun-2016,926.3500,NaN,Mar-2019,699.0000,695.0000,0.0000,NaN,

In [3]:
# before moving to EDA, it is always a good practice in python to lower the column names and remove spaces 
mortgage_raw.columns = mortgage_raw.columns.str.lower().str.replace(' ', '_')
# mortgage_raw.head(2)

# columns of the datasets 
# print(sorted(list(mortgage_raw.columns)))

# to search any column in the dataset we can use the following syntax 
[i for i in mortgage_raw.columns if 'id' in i.lower() or 'loan' in i.lower() or 'status' in i.lower() or 'default' in i.lower()]

['id',
 'member_id',
 'loan_amnt',
 'verification_status',
 'loan_status',
 'initial_list_status',
 'verification_status_joint',
 'hardship_status',
 'hardship_loan_status',
 'settlement_status']

In [ ]:
# defining features in multiples categories
id_cols = ['id', 'issue_d', 'term']
loan_contract_cols = ["loan_amnt", "funded_amnt", "funded_amnt_inv", "int_rate", "installment", "grade", "sub_grade", "purpose", "verification_status"]
borrower_profile_cols = ['annual_inc', 'emp_length', 'emp_title', 'home_ownership', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 
                         'pub_rec', 'revol_bal', 'revol_util', 'total_acc']

outcome_cols = ['loan_status', 'last_pymnt_d', 'last_pymnt_amnt', 'total_rec_prncp', 'total_rec_int', 'recoveries', 'collection_recovery_fee']

hardship_cols = ['hardship_flag', 'hardship_dpd', 'hardship_loan_status', 'debt_settlement_flag', 'settlement_status']

# loan status is the variable using which we can define the default flag
model_cols = id_cols + loan_contract_cols + borrower_profile_cols + ["loan_status"] 

In [26]:
print(hardship_cols)

['hardship_flag', 'hardship_dpd', 'hardship_loan_status', 'debt_settlement_flag', 'settlement_status']


In [5]:
# id is loan_id  and issue_d is issue date (origination date)
# checking the duplicates in loan id
print(mortgage_raw.id.value_counts(dropna=False).unique())

# checking the value counts of the issue dates 
mortgage_raw.issue_d.value_counts(dropna=False).sort_index()

[1]


issue_d
Apr-2008      259
Apr-2009      333
Apr-2010      912
Apr-2011     1563
Apr-2012     3230
            ...  
Sep-2015    28641
Sep-2016    28144
Sep-2017    39713
Sep-2018    39026
NaN            33
Name: count, Length: 140, dtype: int64

In [6]:
# the data is unique on loan_id which is "id" in the dataset 
# converting the issue date to datetime format as month end dates and removing cases where issue date is missing
mortgage_raw = mortgage_raw[~mortgage_raw.issue_d.isna()].copy()
mortgage_raw['t0'] = pd.to_datetime(mortgage_raw['issue_d'], format='%b-%Y') + pd.offsets.MonthEnd(0)
mortgage_raw.t0.value_counts(dropna=False).sort_index()

t0
2007-06-30       24
2007-07-31       63
2007-08-31       74
2007-09-30       53
2007-10-31      105
              ...  
2018-08-31    46079
2018-09-30    39026
2018-10-31    46305
2018-11-30    41973
2018-12-31    40134
Name: count, Length: 139, dtype: int64

**SAS**
```python
* Step 1 - Data Ingestion; 

%let main_dir = /home/u64435593/sasuser.v94/projects;

libname inpdir "&main_dir./data/raw/" access = 'readonly';
libname outdir "&main_dir./data/processed/";

proc import datafile = "&main_dir./data/raw/mortgage_raw_top50.csv" 
out = outdir.mortgage_raw dbms = csv replace; 
guessingrows=max; 
run;

* quick look on the column names and datatypes;

proc contents data = outdir.mortgage_raw; run;


* checing duplicates in loan_id;

proc freq data=outdir.mortgage_raw noprint; tables id / out=loan_id_counts; run;
proc print data=loan_id_counts; where count > 1;run;

proc freq data = outdir.mortgage_raw; tables issue_d/missing norow nocol nocum nopercent; run;

* removing the null issue dates and converting issue date to datetime format as month-end dates;

data outdir.mortgage_raw;
    set outdir.mortgage_raw;
    if not missing(issue_d);
run;

data outdir.mortgage_raw;
    set outdir.mortgage_raw;
    t0 = intnx('month', input(issue_d, monyy7.), 0, 'end');
    format t0 date9.;
run;

proc freq data = outdir.mortgage_raw; tables t0/missing norow nocol nocum nopercent; run;


## **Step 2: Portfolio Definition & Basic Filters**

In [8]:
# loan status 
display(mortgage_raw.loan_status.value_counts(dropna=False).sort_index())

loan_status
Charged Off                                             268559
Current                                                 878317
Default                                                     40
Does not meet the credit policy. Status:Charged Off        761
Does not meet the credit policy. Status:Fully Paid        1988
Fully Paid                                             1076751
In Grace Period                                           8436
Late (16-30 days)                                         4349
Late (31-120 days)                                       21467
Name: count, dtype: int64

In [9]:
# filtering out the active customers only 
# Charged Off, Default, Does not meet the credit policy. Status:Charged Off  are the cases where the loan is defaulted before the PD Horizon
# remaining categories fall under th active loan status 

active_loan_status = [
    'Current',
    'Fully Paid',
    'In Grace Period',
    'Late (16-30 days)',
    'Late (31-120 days)',
    'Does not meet the credit policy. Status:Fully Paid'
]
mortgage_active = mortgage_raw[mortgage_raw.loan_status.isin(active_loan_status)].copy()
display(mortgage_active.loan_status.value_counts(dropna=False).sort_index())

loan_status
Current                                                878317
Does not meet the credit policy. Status:Fully Paid       1988
Fully Paid                                            1076751
In Grace Period                                          8436
Late (16-30 days)                                        4349
Late (31-120 days)                                      21467
Name: count, dtype: int64

**SAS**
```python
proc freq data = outdir.mortgage_raw; tables loan_status/missing norow nocol nocum nopercent; run;

data outdir.mortgage_active;
    set outdir.mortgage_raw;

    if loan_status in (
        "Current",
        "Fully Paid",
        "In Grace Period",
        "Late (16-30 days)",
        "Late (31-120 days)",
        "Does not meet the credit policy. Status:Fully Paid"
    );
run;

proc freq data = outdir.mortgage_active; tables loan_status/missing norow nocol nocum nopercent; run;

## **STEP 3 — PD Target Construction (Retail IRB PD)**

Create a binary PD target (default_flag):
- default_flag = 1 if the loan defaults within 12 months after observation date (T₀)
- default_flag = 0 otherwise

In [ ]:
# 12 month perfomance window 
mortgage_active['t12'] = mortgage_active['t0'] + pd.DateOffset(months=12)


np.int64(0)

In [15]:
mortgage_active.last_pymnt_d.value_counts(dropna=False).sort_index()

last_pymnt_d
Apr-2008       16
Apr-2009       41
Apr-2010       94
Apr-2011      434
Apr-2012      710
            ...  
Sep-2015    10604
Sep-2016    16023
Sep-2017    20309
Sep-2018    25040
NaN           102
Name: count, Length: 137, dtype: int64

In [16]:
mortgage_active.recoveries.value_counts(dropna=False).sort_index()

recoveries
0.0000    1991308
Name: count, dtype: int64

In [17]:
mortgage_active.total_rec_prncp.value_counts(dropna=False).sort_index()


total_rec_prncp
0.0000         105
15.1800          1
17.2700          1
18.2500          1
18.8200          1
              ... 
39989.3400       1
39995.7600       1
39999.4200       1
39999.5300       1
40000.0000    5396
Name: count, Length: 342536, dtype: int64

In [18]:
mortgage_active.collection_recovery_fee.value_counts(dropna=False).sort_index()


collection_recovery_fee
0.0000    1991308
Name: count, dtype: int64

In [19]:
mortgage_active.last_credit_pull_d.value_counts(dropna=False).sort_index()


last_credit_pull_d
Apr-2009       20
Apr-2010       42
Apr-2011      110
Apr-2012      238
Apr-2013      425
            ...  
Sep-2015     2890
Sep-2016     2190
Sep-2017    13167
Sep-2018    29088
NaN            62
Name: count, Length: 142, dtype: int64